In [ ]:
# # CREATES CV FILES IN                        #
#  C:/Users/jozes/Python_Files/_CV_data/     #
# BY SCRAPING SITE BY LOCATIONS AND KEYWORDS #
# @PARAM: LOCATIONS[],KEYWORDS[]             #
# @RETURN: HTML FILE OF CVS                  #
##############################################

import os.path
#import re
import io
import sys
import copy
import time
import random
import requests
import logging
from bs4 import BeautifulSoup
from time import gmtime, strftime
from random import randrange
#from bs4 import SoupStrainer

#print (\"Your system encoding: \"sys.stdout.encoding + '\\n' \"Site indeed.com use standard 'utf-8' encoding\")
#site=\"https://www.google.co.uk/search?q=how+to+pause+in+python&ie=utf-8&oe=utf-8&client=firefox-b-ab&gfe_rd=cr&ei=-NHJWLSnJZL98wePrZrYDg=\"

# set keywords, locations and shuffle them in function shuffle()
keywords_all=['quality+assurance', 'mobile+application','game+designer','health+information','UX+designer',
              'analytics','security+analyst','machine+learning','full stack','salesforce+developer',
              'solutions architect','datacentre','user interface','computing','software',
              'business analyst','digital','automation','microservices','Python',
              'IT+contractor','information+technology','business+analyst','systems+administrator','cybersecurity',
              'web+developer','forensic+computer','data+analyst','database','data+scientist',
              'robotics','network','web','hacker','IT','.NET',
              'Java','Javascript','Python','QA','Ruby',
              'Fidessa','Oracle','Cisco','Linux','Unix',
              'AJAX','deep+learning','big+data','neural+networks','artificial+intelligence',
              'robotics','cybernetics','big+data']
locations_all=['Swindon','Northampton','Coventry','Cardiff','Newcastle upon Tyne',
               'Leeds','Peterborough','Milton Keynes','Brighton','Cambridge',
               'Oxford','Reading','Glasgow','Edinburgh','Bournemouth',
               'Exeter','Hull','Ipswich','Leicester','Greater+London',
               'Middlesbrough','Norwich','Greater+Manchester','Birmingham','Bristol',
               'Bath','Leeds','Southampton','Plymouth','Swansea',
               'Truro','Redruth','Worcester','Malvern','Poole',
               'Nottingham','Sheffield','Bradford','Liverpool','Belfast',
               'Aberdeen','Dundee','Hove','Inverness','Devon',
               'Hove,''Cornwall']
def shuffle(items):  # mutates input list
    print(items)
    i = len(items)
    print("len= "+str(i))
    while i > 1:
        j = randrange(i)  # 0 <= j <= i
        print("randrange= "+str(j))
        item=items[j]
        items[j]=items[i-1]
        items[i-1]=item
        i = i - 1
    print(items)
    return items
keywords=shuffle(keywords_all)
locations=shuffle(locations_all)

site_base="https://www.indeed.com" # site base for the individual links

# output data
all_cv_links=[] # list to be converted to tupple later: keep only unique CV links across different keyword searches
all_cv_dict={}  # dictionary to keep CV_ID and CV_BODY pairs
all_cv_set=()
cv_all_counter=0

# get existing VC ids so we don't get unnecessary dupes
# scraping often gets interrupted by the site management
CV_Set=set()
with open("C:/Users/jozes/Python_Files/_CV_data/MyCvIDs.txt","r",encoding="utf-8") as f:
    for line in f:
        cv_id= line.strip()
        CV_Set.add(cv_id)
#print(CV_Set)
print(len(CV_Set))
# for each keyword, for all avaialbe pages numbered 50*x where x from <1,42>
# 50 pages, each page has 20 CVs
# CVs are uniquely identigied by ID/reference & the results are checked for duplicate CV IDs
# Dictionary contains ID:CV body pairs
# CVs are backed up for future processing in file named after the searched keyword and identified by their ID
# there are expected to be dupes amongst the CVs across the keyword files
# for each location
    
# generating log file name
# current datetime generation
datetime=str(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
datetime=datetime.replace('-','').replace(':','').replace(' ','_')
print("datetime: "+datetime)
log_filename="LOG_"+datetime
#filename=filename.encode(sys.stdout.encoding, errors='replace')
print("Log filename: " + log_filename)
logger=logging.getLogger(log_filename)
format_string = '%(levelname)s: %(asctime)s: %(message)s'
logging.basicConfig(level=logging.DEBUG, filename=log_filename, format=format_string)

empty_location_passes=0
empty_keyword_passes=0
## for each location
for location in locations:
    if empty_keyword_passes==0: 
        empty_location_passes+=1
    if empty_location_passes>3: 
        print("Ended after 3 consecutive empty passes")
        break
    time.sleep(random.randint(1,2))
    # tidying up location string for filename
    location_edit=copy.copy(location)
    location_edit=location_edit.replace("%2C+",'_')
    location_edit=location_edit.replace('+','_')
    print("Location: " + location_edit+'\n')
    logging.info("Location: " + location_edit+'\n')
    is_first_pass=True

    
    ## for each keyword
    cv_region_all_keywords_counter=0
    empty_keyword_passes=0
    for keyword in keywords:
        if empty_keyword_passes>3: break
        site_number=1
        cv_keyword_counter=0

        # tidying up keyword for filename
        keyword_edit=copy.copy(keyword)
        keyword_edit=keyword_edit.replace('+',"")
        print(" Keyword: "+keyword_edit)

        # current datetime generation
        datetime=str(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
        datetime=datetime.replace('-','').replace(':','').replace(' ','_')
        #print("datetime: "+datetime)

        # generating filename from keyword and location strings
        path='C:/Users/jozes/Python_Files/_CV_data/'
        filename='CV_'+keyword_edit+'_'+location_edit+'_'+datetime+".html"
        full_filename=os.path.join(path,filename)
        #filename=filename.encode(sys.stdout.encoding, errors='replace')
        print(" Filename: " + full_filename)

        #logging.debug(\"write this to the log - debug\")
        logging.info("Keyword: "+keyword_edit)
        logging.info("Filename: " + filename)
        logging.info("Location: " + location_edit)
        for_count=0 # debug for loop counter
        ## get site URL
        if is_first_pass:
            site="https://www.indeed.com/resumes?q=\""+keyword+"\"&l="+location+"&lmd=month"
            logging.info("First pass.")
            if location=='Dublin':
                site="https://www.indeed.com/resumes?q="+keyword+"&l=Dublin"
                print('dublin site: '+site)
        else:
            site="https://www.indeed.com/resumes?q=\""+keyword+"\"&l="+location+'&co=GB&start='+"&lmd=month"+str(site_number*50)
            if location=='Dublin':
                site="https://www.indeed.com/resumes?q=\""+keyword+"\"&l="+location+'&co=IE&start='+str(site_number*50)
                print('dublin site: '+site)
            #print('site: '+site)
        
        
        ## while loop through generated sites addresses
        empty_pass=0
        while (site_number<10 and empty_keyword_passes < 3):
            time.sleep(random.randint(2,5))
            print("    Pass: "+str(site_number)+'\n')
            logging.info("Pass: "+str(site_number)+'\n')
            #is_first_pass=False
            # get soup
            try: 
                r=requests.get(site) #(html_doc)
                c=r.content
                #print(str(c))
            except:
                logging.info("Error on pass number "+str(site_number))
                logging.info("Error on site: "+str(site))
                print('exception raised')
            #only_a_tags = SoupStrainer(\"a\")
            soup=BeautifulSoup(c,"html.parser")
            #print("soup: "+soup.get_text())
            #html_doc=open(\"test2.txt\",\"w\")
            ## for each identified link on the page, open CV/link and save it in the dictionary and a file
            while_counter=0 # debug while loop counter
            
            with open(full_filename,"a",encoding="utf-8") as f:
                # all the CV links are identifiable by class=app_link tag/value
                print("empty_pass: "+str(empty_pass))
                if empty_pass>2:
                    print("empty_pass larger than 2: "+str(empty_pass)+"ending sites loop")
                    empty_keyword_passes+=1
                    print("empty_keyword_passes increased to: "+str(empty_keyword_passes))
                    break
                if is_first_pass:
                    f.write(']')
                cv_counter=0
                print("cv_counter@ "+str(cv_counter))
                #print('file opened')
                for link in soup.find_all('a',{"class":"app_link"}): #limit=2
                    inside_for_counter=0 # debug inside for loop counter
                    link_str=link.get('href')
                    #print('str_link@ '+ link_str)
                    # adding links, without the dupes, should not be any in the first page pass
                    link_str_edit=copy.copy(link_str)
                    link_str_edit=link_str_edit.replace("/r/","")
                    link_str_edit=link_str_edit.replace("?sp=0","")
                    link_str_edit=link_str_edit.replace("]","")
                    link_str_edit=link_str_edit.replace("[","")
                    #print("Raw link: "+link_str)
                    #print("Link edit / CV ID: "+link_str_edit)
                    
                    if link_str_edit not in CV_Set:
                        #print("Adding link to all_cv_links.")
                        all_cv_links.append(link_str_edit)
                        #print(link_str+'\n')
                        site=site_base+link_str
                        #print("Site: "+site)
                        #print(site)
                        time.sleep(random.randint(2,5))
                        req=requests.get(site) #(html_doc)
                        con=req.content
                        soup_l=BeautifulSoup(con,"html.parser")
                        #soup[count]=
                        # scrape only CV body
                        cv=soup_l.find_all(id="resume_body")
                        #print("raw cv: "+str(cv))
                        #print("Adding CV "+link_str_edit+" to all_cv_dict. Great job!")
                        #logging.info("Adding CV "+link_str_edit+" to all_cv_dict. Great job!")
                        #all_cv_dict.update({link_str_edit:cv})
                        #file = open('{0}.html'.format(filename),\"w\")
                        #myCV=str(myCV.encode('utf-8'))
                        #print(myCV)
                        print("     Writing cv "+link_str_edit+" to file "+filename)
                        logging.info("Writing cv "+link_str_edit+" to file "+filename)
                        f.write(link_str_edit)
                        #f.write(':')
                        f.write(str(cv))
                        #print("CV:"+str(cv))
                        cv_all_counter+=1
                        cv_counter+=1
                        print("cv_counter@ "+str(cv_counter))
                        logging.info("Increased all cv counter to: "+str(cv_all_counter))
                    else:
                        a=1
                        print("Found dupe: "+link_str)
                        #cv_dupes+=1
                        #cv_counter-=1
            site_number+=1
            if cv_counter<1:
                empty_pass=empty_pass+1
            site="https://www.indeed.com/resumes?q="+keyword+"&l="+location+'&co=GB&start='+str(site_number*50)
            #assert cv_dupes==0, "you have dupes in the first pass"
            #f.close()
            
            is_first_pass=False
            print("Number of all CVs: ", str(cv_all_counter))
            logging.info("Number of all CVs: "+str(cv_all_counter))
#print(all_cv_dict)
#print(soup.get_text())

#print(soup.a.prettify())
#print(\"hi\")
#only_a_tags = SoupStrainer(\"id\")
#only_tags_with_class_app_link = SoupStrainer(rel=\"nofollow\")

#def is_short_string(string):
#    return len(string) < 10
#only_short_strings = SoupStrainer(string=is_short_string)
#print(BeautifulSoup(html_doc, \"html.parser\", parse_only=only_a_tags).prettify())"


['quality+assurance', 'mobile+application', 'game+designer', 'health+information', 'UX+designer', 'analytics', 'security+analyst', 'machine+learning', 'full stack', 'salesforce+developer', 'solutions architect', 'datacentre', 'user interface', 'computing', 'software', 'business analyst', 'digital', 'automation', 'microservices', 'Python', 'IT+contractor', 'information+technology', 'business+analyst', 'systems+administrator', 'cybersecurity', 'web+developer', 'forensic+computer', 'data+analyst', 'database', 'data+scientist', 'robotics', 'network', 'web', 'hacker', 'IT', '.NET', 'Java', 'Javascript', 'Python', 'QA', 'Ruby', 'Fidessa', 'Oracle', 'Cisco', 'Linux', 'Unix', 'AJAX', 'deep+learning', 'big+data', 'neural+networks', 'artificial+intelligence', 'robotics', 'cybernetics', 'big+data']
len= 54
randrange= 39
randrange= 8
randrange= 5
randrange= 26
randrange= 6
randrange= 29
randrange= 30
randrange= 44
randrange= 7
randrange= 2
randrange= 24
randrange= 17
randrange= 33
randrange= 1
ran